In [ ]:
# Google Earth Engine packages
import ee
import geemap
import numpy as np

In [ ]:
# initialize GEE at the beginning of session
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()         # authenticate when using GEE for the first time
    ee.Initialize()

In [ ]:
import configparser
import ast

# read local config.ini file
config = configparser.ConfigParser()
config.read('config.ini')

# get file config from config.ini
dir_output = config['FILE_SETTINGS']['DIR_OUTPUT']
output_gpkg = dir_output + config['FILE_SETTINGS']['GPKG_NAME']

# get date range for forcing data
#date_range = ast.literal_eval(config['CONFIG']['DATE_RANGE'])

In [ ]:
import geopandas as gpd

catchment_new = gpd.read_file(output_gpkg, layer='catchment_new')
catchment = geemap.geopandas_to_ee(catchment_new)

***

In [ ]:
def renameBandName(b):
    split = ee.String(b).split('_')   
    return ee.String(split.splice(split.length().subtract(2),1).join("_"))


def buildFeature(i):
    t1 = startDate.advance(i,'day')
    t2 = t1.advance(1,'day')
    #feature = ee.Feature(point)
    dailyColl = collection.filterDate(t1, t2)
    dailyImg = dailyColl.toBands()
    # renaming and handling names
    bands = dailyImg.bandNames()
    renamed = bands.map(renameBandName)
    # Daily extraction and adding time information
    dict = dailyImg.rename(renamed).reduceRegion(
      reducer=ee.Reducer.mean(),
      geometry=catchment,
    ).combine(
      ee.Dictionary({'system:time_start':t1.millis(),'isodate':t1.format('YYYY-MM-dd')})
    )
    return ee.Feature(None,dict)


def getImageCollection(var):
    collection = ee.ImageCollection('NASA/GDDP-CMIP6')\
        .select(var)\
        .filterDate(startDate, endDate)\
        .filterBounds(catchment)
    return collection


def getTask(fileName):
    task = ee.batch.Export.table.toDrive(**{
      'collection': ee.FeatureCollection(ee.List.sequence(0,n).map(buildFeature)),
      'description':fileName,
      'fileFormat': 'CSV'
    })
    return task

# Download scenarios

In [ ]:
startDate = ee.Date('2020-01-01')
endDate = ee.Date('2021-01-31')
n = endDate.difference(startDate,'day').subtract(1)

In [ ]:
collection = getImageCollection('tas')
task_tas_ssp = getTask('CMIP6_tas_ssp')
task_tas_ssp.start()

collection = getImageCollection('pr')
task_pr_ssp = getTask('CMIP6_pr_ssp')
task_pr_ssp.start()

print('Tasks for scenarios started...')

# Download historical data

In [ ]:
startDate = ee.Date('2010-01-01')
endDate = ee.Date('2011-01-31')
n = endDate.difference(startDate,'day').subtract(1)

In [ ]:
collection = getImageCollection('tas')
task_tas_hist = getTask('CMIP6_tas_hist')
task_tas_hist.start()

collection = getImageCollection('pr')
task_pr_hist = getTask('CMIP6_pr_hist')
task_pr_hist.start()

print('Tasks for historical data started...')

In [ ]:
import time

while task_tas_ssp.active() or task_pr_ssp.active() or task_tas_hist.active() or task_pr_hist.active():
    print(".", end = '')
    time.sleep(2)
    
print('done.')